# TSP + QAOA - naive approach

In the previous tutorials we learned what is the Traveling Salesman Problem and how to use QAOA to solve optimization problems.
<br>
Now it's finally time to solve TSP using quantum computing!

In this tutorial you will learn the following:

- How to encode TSP for QAOA

## Encoding the problem

One of the important decisions we need to make with any problem we encounter is how to encode our input and output data. What will be the good choice depends on many factors:
- what algorithm we use?
- do we have any constraints (e.g. amount of data, memory)?
- what hardware do we use?
- many others

One of the most convenient way of encoding solution to TSP is the one we used in the first part:

0 -> 1 -> 2 -> 3 , which for Python translates into `[0,1,2,3]` or `(0,1,2,3)`.

This means that we start from the city 0, then go to 1 and so on.

However, in case of quantum computers, one of the main constraints is that we need to use binary representation. Qubits can take only 0 or 1 values during the measurement, so this format won't work.

We know that at any given time we can be only in one city. Hence, for N cities we have N time slots.
<br>
Using binary representation, we could use N strings (representing time slots) of N bits (representing cities), where the active bit in each string is the city that we visit.

So in the case of [0,1,2,3] we will get the following matrix:

$\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}$

Which translates into the following string of bits:

`[1,0,0,0, 0,1,0,0, 0,0,1,0, 0,0,0,1]`


TODO: write about the problems with this encoding later

This is a little bit unwieldy, so we will use the following helper methods:

In [29]:
import numpy as np

def points_order_to_binary_state(points_order):
    """
    Transforms the order of points from the standard representation: [0, 1, 2],
    to the binary one: [1,0,0,0,1,0,0,0,1]
    """
    number_of_points = len(points_order)
    binary_state = np.zeros((len(points_order))**2)
    for j in range(len(points_order)):
        p = points_order[j]
        binary_state[(number_of_points) * (j) + (p)] = 1
    return binary_state

def binary_state_to_points_order(binary_state):
    """
    Transforms the the order of points from the binary representation: [1,0,0,0,1,0,0,0,1],
    to the binary one: [0, 1, 2]
    """
    points_order = []
    number_of_points = int(np.sqrt(len(binary_state)))
    for p in range(number_of_points):
        for j in range(number_of_points):
            if binary_state[(number_of_points) * p + j] == 1:
                points_order.append(j)
    return points_order

Let's see if it works on an example:

In [40]:
points_order = [0, 1, 2]
binary_state = points_order_to_binary_state(points_order)
points_order_again = binary_state_to_points_order(binary_state)

print("Binary representation of ", points_order, "is", binary_state)
print(points_order_again, "should be equal to", points_order)

Binary representation of  [0, 1, 2] is [1. 0. 0. 0. 1. 0. 0. 0. 1.]
[0, 1, 2] should be equal to [0, 1, 2]


## Quick overview 